In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

import dask
dask.config.set({"dataframe.convert-string": False})
import dask.dataframe as dd

## Default + WRDS Factors

In [2]:
merged_df_original = pd.read_sas("merged_df.sas7bdat", encoding='ISO-8859-1')
signals_original = pd.read_sas("signals_raw_plus.sas7bdat", encoding='ISO-8859-1')

In [3]:
merged_df = merged_df_original.copy()
signals = signals_original.copy()

In [4]:
permnos = merged_df['permno'].unique()
permno_to_gvkey = merged_df.set_index('permno')['gvkey'].to_dict()

In [5]:
# convert yyyymm to datetime
merged_df['date'] = pd.to_datetime(merged_df['yyyymm'].astype(int).astype(str), format='%Y%m')
merged_df['mktcap'] = merged_df['PRC'] * merged_df['SHROUT'] / 1000

In [6]:
len(merged_df['permno'].unique())

1497

In [7]:
january_filter = (merged_df['date'].dt.month == 1)
january_data = merged_df[january_filter]

# Group by permno and check if any January data point has mkt_val or mktcap < 100
valid_permnos = january_data.groupby('permno').apply(
    lambda group: ((group['PRC'] > 5) & (group['mktcap'] >= 100)).all()
)
valid_permnos = valid_permnos[valid_permnos].index

# Filter the dataframe to include only the valid permnos
merged_df = merged_df[merged_df['permno'].isin(valid_permnos)]

merged_df.head()

/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_42778/3874959622.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  valid_permnos = january_data.groupby('permno').apply(


,permno,yyyymm,monthid,ticker,conm,gvkey,cusip,naics,gsubind,IM,...,ret_f5,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,date,mktcap
1224,10104.0,198602.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.364103,0.064516,-0.136364,0.263158,0.194444,-0.034884,0.301205,0.425926,1986-02-01,NaN
1225,10104.0,198603.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.064516,-0.136364,0.263158,0.194444,-0.034884,0.301205,0.425926,0.142857,1986-03-01,275.320375
1226,10104.0,198604.0,76.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,0.636488,...,-0.136364,0.263158,0.194444,-0.034884,0.301205,0.425926,0.142857,0.068182,1986-04-01,329.725000
1227,10104.0,198605.0,77.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,NaN,...,0.263158,0.194444,-0.034884,0.301205,0.425926,0.142857,0.068182,0.159574,1986-05-01,309.941500
1228,10104.0,198606.0,78.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,NaN,...,0.194444,-0.034884,0.301205,0.425926,0.142857,0.068182,0.159574,-0.183486,1986-06-01,321.481875


In [8]:
signals.rename(columns={'PERMNO':'permno'}, inplace=True)
signals['yyyymm'] = signals['fdate'].dt.strftime('%Y%m').astype(int)

In [9]:
merged_df = pd.merge(
    merged_df, 
    signals, 
    on=['yyyymm', 'permno'], 
    how='inner',
    suffixes=('', '_signals')
)

In [10]:
missing_percentage = merged_df.groupby('permno').apply(
    lambda group: group.isnull().mean() * 100
)
# filter out companies that dont have 0% missing data for ticker in merged_df, meaning they might have been delisted or are newly listed
valid_permnos = missing_percentage[missing_percentage['ticker'] == 0].index
merged_df = merged_df[merged_df['permno'].isin(valid_permnos)]

# for each permno, check if it has data for yyyymm starting from 199501
valid_permnos = merged_df.groupby('permno')['yyyymm'].min().reset_index()
valid_permnos = valid_permnos[valid_permnos['yyyymm'] <= 199501]['permno']
merged_df = merged_df[merged_df['permno'].isin(valid_permnos)]

# for each permno, check if it has data for yyyymm that ends in 201912
valid_permnos = merged_df.groupby('permno')['yyyymm'].max().reset_index()
valid_permnos = valid_permnos[valid_permnos['yyyymm'] >= 201912]['permno']
merged_df = merged_df[merged_df['permno'].isin(valid_permnos)]
print(valid_permnos.shape[0])

272


/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_42778/1942721390.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_percentage = merged_df.groupby('permno').apply(


## Mean Reversion Factor

In [11]:
# Create a mean reversion signal by calculating rolling z-scores of returns
def calculate_z_score(series):
    # Convert numpy array to pandas Series if needed
    if isinstance(series, np.ndarray):
        series = pd.Series(series)
    
    if len(series) == 0 or series.isna().all():
        return np.nan
    mean = series.mean()
    std = series.std()
    # Handle division by zero
    if std == 0:
        return np.nan
    # Return the z-score of the last value in the series
    return (series.iloc[-1] - mean) / std

# Sort dataframe by permno and date for proper time series analysis
merged_df = merged_df.sort_values(['permno', 'yyyymm'])

# Calculate returns by permno
merged_df['ret'] = merged_df.groupby('permno')['PRC'].pct_change()

# Create rolling z-scores by permno
z_scores = []
for permno, group in merged_df.groupby('permno'):
    group = group.sort_values('yyyymm')
    group['rolling_z_score'] = group['ret'].rolling(window=36).apply(calculate_z_score, raw=False)
    z_scores.append(group)

# Combine results
merged_df = pd.concat(z_scores)

# Clean up z-scores
merged_df['rolling_z_score'] = merged_df['rolling_z_score'].fillna(0)
merged_df['rolling_z_score'] = merged_df['rolling_z_score'].replace([np.inf, -np.inf], 0)

# Create mean reversion signal
merged_df['mean_reversion_signal'] = np.where(
    merged_df['rolling_z_score'] > 1, -1,
    np.where(merged_df['rolling_z_score'] < -1, 1, 0)
)

## Macro Uncertainty

In [12]:
macro_uncertainty_original = pd.read_sas("macro.sas7bdat", encoding='ISO-8859-1')

In [13]:
macro_uncertainty = macro_uncertainty_original.copy()
macro_uncertainty["yyyymm"] = macro_uncertainty["date"].dt.strftime('%Y%m').astype(int)
macro_uncertainty.set_index("yyyymm", inplace=True)

merged_df = pd.merge(
    merged_df,
    macro_uncertainty,
    left_on="yyyymm",
    right_index=True,
    how="left",
) 

## SEC Filings Sentiment Factors

In [14]:
sec_sentiment_analytics_original = pd.read_sas("sec_filings_sentiments.sas7bdat", encoding='ISO-8859-1')

In [15]:
sec_sentiment_analytics = sec_sentiment_analytics_original.copy()
sec_sentiment_analytics.columns = sec_sentiment_analytics.columns.str.lower()
sec_sentiment_analytics.head()

,cik,fdate,gvkey,form,word_count,complex_word_count,average_words_per_sentence,lm_negative_count,lm_positive_count
0,0000001800,2010-09-21,001078,3,316.0,55.0,35.111111,2.0,0.0
1,0000001800,2007-02-01,001078,4,121.0,17.0,8.642857,1.0,1.0
2,0000001800,2005-04-26,001078,4,72.0,12.0,18.000000,1.0,0.0
3,0000001800,2008-02-20,001078,4,212.0,35.0,11.157895,1.0,1.0
4,0000001800,2017-03-02,001078,4,24.0,4.0,8.000000,0.0,0.0


In [16]:
sec_sentiment_analytics.rename(columns={'PERMNO':'permno'}, inplace=True)
sec_sentiment_analytics = sec_sentiment_analytics[sec_sentiment_analytics['form'].isin(['10-K', '10-Q', '8-K'])]
sec_sentiment_analytics['yyyymm'] = sec_sentiment_analytics['fdate'].dt.strftime('%Y%m').astype(int)
gvkey_to_permno = merged_df.set_index('gvkey')['permno'].to_dict()
sec_sentiment_analytics['permno'] = sec_sentiment_analytics['gvkey'].map(gvkey_to_permno)

# Hypothesis: Higher negative intensity in SEC filings associated with underperformance
sec_sentiment_analytics['negative intensity'] = sec_sentiment_analytics['lm_negative_count'] / sec_sentiment_analytics['word_count']
# Hypothesis: Higher fog score in SEC filings associated with uncertainty and underperformance
sec_sentiment_analytics['fog score'] = sec_sentiment_analytics['complex_word_count'] * sec_sentiment_analytics['average_words_per_sentence']
# Hypothesis: Higher polarity ratio in SEC filings associated with optimism and overperformance
sec_sentiment_analytics['polarity ratio'] = sec_sentiment_analytics['lm_positive_count'] / (sec_sentiment_analytics['lm_negative_count'] + 1)
sec_sentiment_analytics.set_index('yyyymm', inplace=True)
sec_sentiment_analytics.head().round(4)

,cik,fdate,gvkey,form,word_count,complex_word_count,average_words_per_sentence,lm_negative_count,lm_positive_count,permno,negative intensity,fog score,polarity ratio
yyyymm,,,,,,,,,,,,,
201405,0000001800,2014-05-07,001078,10-Q,88357.0,25888.0,29.2864,1101.0,862.0,20482.0,0.0125,758165.7328,0.7822
200709,0000001800,2007-09-19,001078,8-K,8525.0,1658.0,39.6512,59.0,25.0,20482.0,0.0069,65741.6279,0.4167
201601,0000001800,2016-01-28,001078,8-K,4986.0,1112.0,24.9300,48.0,34.0,20482.0,0.0096,27722.1600,0.6939
200701,0000001800,2007-01-24,001078,8-K,1166.0,290.0,26.5000,16.0,5.0,20482.0,0.0137,7685.0000,0.2941
201401,0000001800,2014-01-22,001078,8-K,5610.0,1213.0,27.3659,59.0,58.0,20482.0,0.0105,33194.7805,0.9667


In [17]:
pivoted_sentiment = sec_sentiment_analytics.pivot_table(
    index=['yyyymm'],
    columns='form',
    values=['negative intensity', 'fog score', 'polarity ratio']
).reset_index()

pivoted_sentiment.columns = ['_'.join(col).strip() for col in pivoted_sentiment.columns.values]
pivoted_sentiment.rename(columns={'yyyymm_': 'yyyymm'}, inplace=True)

pivoted_sentiment['yyyymm'] = pd.to_numeric(pivoted_sentiment['yyyymm'], errors='coerce')

pivoted_sentiment.head().round(4)

,yyyymm,fog score_10-K,fog score_10-Q,fog score_8-K,negative intensity_10-K,negative intensity_10-Q,negative intensity_8-K,polarity ratio_10-K,polarity ratio_10-Q,polarity ratio_8-K
0,199501,110134.6192,105942.2335,22011.2837,0.0063,0.0109,0.0108,1.2152,1.6336,1.0295
1,199502,348144.7337,93458.0088,100140.2401,0.0120,0.0089,0.0096,0.6541,1.0460,0.9310
2,199503,294058.2757,16166.1687,115545.4908,0.0104,0.0061,0.0075,0.7178,1.2160,0.8233
3,199504,123798.7971,23057.2210,62389.0549,0.0073,0.0110,0.0097,0.9776,0.7984,1.2643
4,199505,268684.8363,51797.6449,94526.0997,0.0097,0.0112,0.0107,0.7401,0.8140,1.1186


In [18]:
pivoted_sentiment_dd = dd.from_pandas(pivoted_sentiment)
merged_df_dd = dd.from_pandas(merged_df)

# Conversion to matching dtypes
merged_df_dd['yyyymm'] = merged_df_dd['yyyymm'].astype('int64')

pivoted_sentiment_dd['yyyymm'] = pivoted_sentiment_dd['yyyymm'].astype('int64')

# Persist the DataFrames in memory to reduce computation graph size
pivoted_sentiment_dd = pivoted_sentiment_dd.persist()
merged_df_dd = merged_df_dd.persist()

# Perform the merge
merged_df_dd = dd.merge(
    merged_df_dd,
    pivoted_sentiment_dd,
    on='yyyymm',
    how='left'
)

# Compute the result
merged_df = merged_df_dd.compute()
merged_df.head()

,permno,yyyymm,monthid,ticker,conm,gvkey,cusip,naics,gsubind,IM,...,h_12,fog score_10-K,fog score_10-Q,fog score_8-K,negative intensity_10-K,negative intensity_10-Q,negative intensity_8-K,polarity ratio_10-K,polarity ratio_10-Q,polarity ratio_8-K
0,10104.0,199501,181.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,-0.015804,...,0.873199,110134.619154,105942.233549,22011.283671,0.006251,0.010870,0.010757,1.215169,1.633568,1.029470
1,10104.0,199502,182.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,-0.034445,...,0.875654,348144.733715,93458.008827,100140.240133,0.011994,0.008880,0.009647,0.654146,1.045985,0.931036
2,10104.0,199503,183.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,-0.007065,...,0.874511,294058.275684,16166.168654,115545.490783,0.010384,0.006102,0.007517,0.717843,1.215951,0.823326
3,10104.0,199504,184.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,0.005445,...,0.873917,123798.797080,23057.220964,62389.054887,0.007283,0.011033,0.009737,0.977610,0.798445,1.264298
4,10104.0,199505,185.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,0.031876,...,0.872138,268684.836310,51797.644942,94526.099682,0.009725,0.011173,0.010660,0.740053,0.814023,1.118629


## Winsorization & Standardization

In [19]:
# Uncomment and run to debug non data columns and/or sanity check data?
# merged_df.head(100).to_excel('df_before_standardizing.xlsx')

In [20]:
non_factor_cols = {'permno', 'yyyymm', 'monthid', 'ticker', 'conm', 'gvkey', 'cusip', 'naics', 'gsubind', 'PRC', 'RET', 'SHROUT', 'ret_f1', 'ret_f2', 'ret_f3', 'ret_f4', 'ret_f5', 'ret_f6', 'ret_f7', 'ret_f8', 'ret_f9', 'ret_f10', 'ret_f11', 'ret_f12', 'date_x', 'fdate', 'ret', 'date_y', 'date'}
factor_cols = set(merged_df.columns) - non_factor_cols

In [21]:
# Winsorize
for column in factor_cols:
    lower_bounds = merged_df.groupby('monthid')[column].transform('quantile', 0.01)
    upper_bounds = merged_df.groupby('monthid')[column].transform('quantile', 0.99)
    merged_df[column] = merged_df[column].clip(lower=lower_bounds, upper=upper_bounds)
    
merged_df.head()

,permno,yyyymm,monthid,ticker,conm,gvkey,cusip,naics,gsubind,IM,...,h_12,fog score_10-K,fog score_10-Q,fog score_8-K,negative intensity_10-K,negative intensity_10-Q,negative intensity_8-K,polarity ratio_10-K,polarity ratio_10-Q,polarity ratio_8-K
0,10104.0,199501,181.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,-0.015804,...,0.873199,110134.619154,105942.233549,22011.283671,0.006251,0.010870,0.010757,1.215169,1.633568,1.029470
1,10104.0,199502,182.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,-0.034445,...,0.875654,348144.733715,93458.008827,100140.240133,0.011994,0.008880,0.009647,0.654146,1.045985,0.931036
2,10104.0,199503,183.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,-0.007065,...,0.874511,294058.275684,16166.168654,115545.490783,0.010384,0.006102,0.007517,0.717843,1.215951,0.823326
3,10104.0,199504,184.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,0.005445,...,0.873917,123798.797080,23057.220964,62389.054887,0.007283,0.011033,0.009737,0.977610,0.798445,1.264298
4,10104.0,199505,185.0,ORCL,ORACLE CORP,012142,68389X105,519130,45103020,0.031876,...,0.872138,268684.836310,51797.644942,94526.099682,0.009725,0.011173,0.010660,0.740053,0.814023,1.118629


In [22]:
def categorize_factors(df, factor_columns, skew_threshold=0.5, kurt_threshold=3.0, min_samples=8):
    """
    Automatically categorizes factors based on their statistical properties.
    
    Args:
        df: DataFrame with your factor data
        factor_columns: List of column names for factors
        skew_threshold: Absolute skewness threshold to consider a distribution skewed
        kurt_threshold: Kurtosis threshold beyond normal (normal = 3)
        min_samples: Minimum number of samples required to calculate statistics
    
    Returns:
        Dictionary with 'zscore_factors' and 'percentile_factors' lists
    """
    zscore_factors = []
    percentile_factors = []
    
    for factor in factor_columns:
        monthly_skews = []
        monthly_kurts = []
        
        for _, group in df.groupby('yyyymm'):
            valid_data = group[factor].dropna()
            
            # Only consider distribution if we have enough samples
            if len(valid_data) >= min_samples:
                monthly_skews.append(skew(valid_data))
                monthly_kurts.append(kurtosis(valid_data, fisher=False))
        
        if monthly_skews and monthly_kurts:
            avg_skew = sum(monthly_skews) / len(monthly_skews)
            avg_kurt = sum(monthly_kurts) / len(monthly_kurts)
            
            # If data is normally distributed, use z-scores, otherwise percentile
            if (abs(avg_skew) > skew_threshold or 
                avg_kurt > kurt_threshold + 3):
                percentile_factors.append(factor)
            else:
                zscore_factors.append(factor)
        else:
            # Default behavior if not enough data = percentile
            percentile_factors.append(factor)
    
    return {
        'zscore_factors': zscore_factors,
        'percentile_factors': percentile_factors
    }

factor_categories = categorize_factors(merged_df, factor_cols)
factor_categories

/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_42778/1626272597.py:27: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  monthly_skews.append(skew(valid_data))
/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_42778/1626272597.py:28: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  monthly_kurts.append(kurtosis(valid_data, fisher=False))


{'zscore_factors': ['xret_5',
  'CAPES',
  'IM',
  'KDJ_120',
  'CEI',
  'roa_q',
  'xret_10',
  'momentum',
  'CF',
  'deviation_pct120',
  'MoneyFlowIndex_20',
  'h_3',
  'RSI_120',
  'deviation_pct20',
  'xret_40',
  'LiqVol',
  'sue',
  'seasonality',
  'Skew_OTM',
  'h_1',
  'polarity ratio_10-Q',
  'O_score',
  'xret_120',
  'FIRMTANG',
  'h_12',
  'xret_20',
  'log_vol_dollar_20',
  'leverage',
  'negative intensity_8-K',
  'trend_factor',
  'polarity ratio_10-K',
  'negative intensity_10-K',
  'fog score_10-K',
  'rolling_z_score',
  'DBREADTH',
  'log_vol_dollar_120',
  'RSI_20',
  'lag_log_size',
  'PFS_Score',
  'xret_indsize_120',
  'FCF',
  'KDJ_20',
  'negative intensity_10-Q',
  'fog score_8-K',
  'polarity ratio_8-K',
  'profitability',
  'str_mod',
  'sue_NI',
  'MoneyFlowIndex_120',
  'Accrual',
  'dBlock_N',
  'xret_indsize_20',
  'fog score_10-Q',
  'roa',
  'mean_reversion_signal'],
 'percentile_factors': ['CF_P',
  'SG',
  'ROAG',
  'xret_indsize_std20',
  'NOA',


In [23]:
# Kept getting spammed with 'highly fragmented' warning. So I fed the code into AI until the warning went away.
# This code looks hella weird to me but hey at least it's warning free :)

transformed_data = {}

for factor in factor_categories['zscore_factors']:
    transformed_data[factor] = merged_df.groupby(['yyyymm'])[factor].transform(
        lambda x: (x - x.mean()) / x.std() if x.std() != 0 else 0
    )

for factor in factor_categories['percentile_factors']:
    transformed_data[factor] = merged_df.groupby(['yyyymm'])[factor].transform(
        lambda x: x.rank(pct=True)
    )

merged_df[list(transformed_data.keys())] = pd.DataFrame(transformed_data, index=merged_df.index)

In [24]:
merged_df.to_parquet('cleaned.parquet')

In [25]:
# Uncomment if you want to see in Excel as well
# merged_df.to_excel('cleaned.xlsx')